In [66]:
import pickle
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
# ./test/test_texts.dat
# ./train/train_labels.txt
# ./train/train_texts.dat

In [68]:
file_name = './train/train_texts.dat'
with open(file_name,'rb') as f:
    train_texts = pickle.load(f)

In [69]:
file_name1 = './train/train_labels.txt'
# train_labels = np.loadtxt(file_name1)
train_label = pd.read_table(file_name1,header=None)


In [70]:


vectorizer = TfidfVectorizer(max_features = 10000)
vectors_train = vectorizer.fit_transform(train_texts)

In [71]:
vectors_train = vectors_train.toarray()
vectors_train = pd.DataFrame(vectors_train)
vectors_train.shape

(11314, 10000)

In [72]:
num_train = int(0.8 * vectors_train.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( vectors_train.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

# 提取训练集和验证集的特征
train_features = vectors_train.iloc[train_indices].values.astype(np.float32)
train_features = torch.from_numpy(train_features)
valid_features = vectors_train.iloc[valid_indices].values.astype(np.float32)
valid_features = torch.from_numpy(valid_features)
train_valid_features = vectors_train[:vectors_train.shape[0]].values.astype(np.float32)
train_valid_features = torch.from_numpy(train_valid_features)

# 提取训练集和验证集的label
train_labels = train_label.values[train_indices].astype(np.float32)
train_labels = torch.from_numpy(train_labels)
valid_labels = train_label.values[valid_indices].astype(np.float32)
valid_labels = torch.from_numpy(valid_labels)
train_valid_labels = train_label.values[:, None].astype(np.float32)
train_valid_labels = torch.from_numpy(train_valid_labels)

In [73]:
train_labels.shape

torch.Size([9051, 1])

In [76]:

# 法一： 只用一个输出
def get_model(feat_dim,output_dim = 1):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim/2)),
        nn.BatchNorm1d(int(feat_dim/2)),
        nn.ReLU(inplace=True),
        nn.Linear(int(feat_dim/2),int(feat_dim/4)),
        nn.BatchNorm1d(int(feat_dim/4)),
        nn.ReLU(inplace=True),
        nn.Linear(int(feat_dim/4), output_dim),
        nn.Softmax(output_dim,20)
        )
    return net
# 法二： 20维 后加一个 softmax
'''def Net(nn.Module):
    def __init__(self):
        super(Net.self).__init()
        self.L1 = nn.Linear(10000,7500)
        self.L2 = nn.Linear(7500,2500)
        self.L3 = nn.Linear(2500,500)
        self.L4 = nn.Linear(500,100)
        self.L5 = nn.Linear(100,20)
        
    def forward(self,x):
'''        
        
        
        
        
        
        

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        print(output)
        print('epoch:{}'.format(e))
        print('accuracy:{}'.format(accu(output,y_valid)))
            

In [77]:
def accu(a,b):
    c = a.numpy()
    d = b.numpy()
    num = d.shape[0]
    count = 0
    for i in range(num):
        if c[i] == d[i]:
            count += 1
    return count/num

In [78]:
a=np.array([[11.6803],
        [ 9.3713],
        [12.7842],
        ...,
        [ 1.1657],
        [ 4.4483],
        [ 2.6290]])

In [79]:
# 可以调整的超参
batch_size = 128
epochs = 100
lr = 0.01
wd = 0
use_gpu = False
feat_dim = vectors_train.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_labels, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)

TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given